In [1]:
# 5.3.1 多输⼊通道 
# 实现含多个输⼊通道的互相关运算。我们只需要对每个通道做互相关运算，然后通 过add_n函数来进⾏累加。 
import d2lzh as d2l
from mxnet import nd

def corr2d_multi_in(X,K):
    # ⾸先沿着X和K的第0维（通道维）遍历（for x,k in zip(X,K)得到每个输入通道上的输入数组x,核数组k）。
    # d2l.corr2d(x,k)得到每个输入通道上的输出数组
    # 然后使⽤*将输出数组列表变成add_n函数的位置参数 （positional argument）来进⾏相加
    return nd.add_n(*[d2l.corr2d(x,k) for x,k in zip(X,K)])

In [2]:
X=nd.array([ [ [0,1,2],[3,4,5],[6,7,8] ],
             [ [1,2,3],[4,5,6],[7,8,9] ] ])
K=nd.array([ [ [0,1],[2,3] ],
             [ [1,2],[3,4] ] ])
corr2d_multi_in(X,K)


[[ 56.  72.]
 [104. 120.]]
<NDArray 2x2 @cpu(0)>

In [3]:
# 5.3.2 多输出通道 
# 实现⼀个互相关运算函数来计算多个通道的输出。 
def corr2d_multi_in_out(X,K):
    # 每个输出通道上的结果由卷积核在该输出通道上的核数组(k)与整个输⼊数组(X)计算而来。 
    # for k in K 得到每个输出通道上的核数组k，corr2d_multi_in(X,k)的得到每个输出通道的输出数组的合并在一起
    # 通过nd.stack(*)将每个输出通道的输出数组的合并在一起
    return nd.stack(*[corr2d_multi_in(X,k) for k in K])

In [4]:
x=nd.array([1,2,3]).reshape(1,1,3)
y=nd.array([4,5,6]).reshape(1,1,3)
z=nd.array([7,8,9]).reshape(1,1,3)
nd.stack(x,y,z,axis=0)


[[[[1. 2. 3.]]]


 [[[4. 5. 6.]]]


 [[[7. 8. 9.]]]]
<NDArray 3x1x1x3 @cpu(0)>

In [5]:
K=nd.stack(K,K+1,K+2)
K.shape

(3, 2, 2, 2)

In [6]:
corr2d_multi_in_out(X,K)


[[[ 56.  72.]
  [104. 120.]]

 [[ 76. 100.]
  [148. 172.]]

 [[ 96. 128.]
  [192. 224.]]]
<NDArray 3x2x2 @cpu(0)>

In [7]:
# 5.3.3 1×1卷积层 
# 1×1卷积层被当作保持⾼和宽维度形状不变的全连接层使⽤。
# 输出中的每个元素来⾃输⼊中在⾼和宽上相同位置的元素在不同通 道之间的按权重累加。
def corr2d_multi_in_out_1x1(X,K):
    c_i,h,w=X.shape # 输入通道数，高，宽
    c_o=K.shape[0] # 输出通道数
    X=X.reshape((c_i,h*w)) #一个通道一个h*w的数组
    K=K.reshape((c_o,c_i))
    Y=nd.dot(K,X) # 全连接层的矩阵乘法 
    return Y.reshape((c_o,h,w))

In [10]:
X=nd.random.uniform(shape=(3,3,3))
K=nd.random.uniform(shape=(2,3,1,1))

Y1=corr2d_multi_in_out_1x1(X,K)
Y2=corr2d_multi_in_out(X,K)

(Y1-Y2).norm().asscalar()<1e-6

True

In [8]:
d2l.corr2d??
zip??
nd.add_n??
nd.stack??